In [5]:
import pandas as pd
import numpy as np

In [6]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [7]:
df.shape

(12437, 20)

In [8]:
df.sample(4)

,Facing,No_Bedroom,No_Bathroom,No_Balcony,Corner_Property,Furnishing,study room,servant room,store room,pooja room,Overlooking_main road,Overlooking_club,Overlooking_pool,Overlooking_NA,flat_age,sector_num,Super Built-up Area,luxury_category,floor_category,price
6101,4.0,3,3.0,2,1,2.0,1,0,0,1,1,1,1,0,3,47,1770.0,1.0,0.0,1.30
1687,0.0,2,2.0,3,1,1.0,1,0,0,0,1,1,1,0,3,68,1450.0,1.0,1.0,1.20
11471,4.0,3,5.0,3,1,1.0,0,1,0,1,1,0,0,0,3,70,1950.0,2.0,1.0,2.25
7826,4.0,4,4.0,3,0,1.0,0,0,0,0,1,1,1,0,1,37,2191.0,0.0,0.0,2.46


In [9]:
df['Furnishing'].value_counts()

Furnishing
1.0    9686
0.0    2220
2.0     531
Name: count, dtype: int64

In [10]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12437 entries, 0 to 12436
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Facing                 12437 non-null  float64
 1   No_Bedroom             12437 non-null  int64  
 2   No_Bathroom            12437 non-null  float64
 3   No_Balcony             12437 non-null  int64  
 4   Corner_Property        12437 non-null  int64  
 5   Furnishing             12437 non-null  float64
 6   study room             12437 non-null  int64  
 7   servant room           12437 non-null  int64  
 8   store room             12437 non-null  int64  
 9   pooja room             12437 non-null  int64  
 10  Overlooking_main road  12437 non-null  int64  
 11  Overlooking_club       12437 non-null  int64  
 12  Overlooking_pool       12437 non-null  int64  
 13  Overlooking_NA         12437 non-null  int64  
 14  flat_age               12437 non-null  int64  
 15  se

In [12]:
df['No_Balcony'].value_counts()

No_Balcony
4    4945
3    4351
2    2545
1     582
0      14
Name: count, dtype: int64

In [13]:
balcony_mapping = {
    4: '3+',
    3 : '3',
    2 : '2',
    1 : '1',
    0 : '0',
}

df['No_Balcony'] = df['No_Balcony'].map(balcony_mapping)

In [14]:
df['No_Balcony'].value_counts()

No_Balcony
3+    4945
3     4351
2     2545
1      582
0       14
Name: count, dtype: int64

In [15]:
df['flat_age'].value_counts()

flat_age
3    5441
1    2267
4    2123
2    1859
5     743
6       4
Name: count, dtype: int64

In [16]:
status_mapping = {
    3 : 'Relatively New',
    1 : 'New Property',
    4: 'Moderately Old',
    2: 'Under Construction',
    5 : 'Old Property',
    6: 'Undefined'
}

df['flat_age'] = df['flat_age'].replace(status_mapping)

In [17]:
df['flat_age'].value_counts()

flat_age
Relatively New        5441
New Property          2267
Moderately Old        2123
Under Construction    1859
Old Property           743
Undefined                4
Name: count, dtype: int64

In [18]:
df['Corner_Property'].value_counts()

Corner_Property
1    8204
0    4233
Name: count, dtype: int64

In [19]:
replacements = {
    1 : 'Yes',
    0 : 'No'
}

In [20]:
df['Corner_Property'] = df['Corner_Property'].replace(replacements)

In [21]:
df['Corner_Property'].value_counts()

Corner_Property
Yes    8204
No     4233
Name: count, dtype: int64

In [22]:
df['Facing'].value_counts()

Facing
4.0    4351
0.0    4020
2.0    1809
1.0     760
5.0     557
3.0     409
6.0     338
7.0     193
Name: count, dtype: int64

In [23]:
status_mapping = {
        
    0.0 : 'East',
    1.0 : 'West',
    2.0 : 'North',
    3.0 : 'South',
    4.0 : 'North-East',
    5.0 : 'North-West',
    6.0 : 'South-East',
    7.0 : 'South-West',
}

df['Facing'] = df['Facing'].replace(status_mapping)

In [24]:
df['Facing'].value_counts()

Facing
North-East    4351
East          4020
North         1809
West           760
North-West     557
South          409
South-East     338
South-West     193
Name: count, dtype: int64

In [25]:
df['luxury_category'].value_counts()

luxury_category
2.0    6581
1.0    4982
0.0     874
Name: count, dtype: int64

In [26]:
df['luxury_category'] = df['luxury_category'].replace({0.0:'Budget',1.0:'Standard',2.0:'Luxury'})

In [27]:
df['luxury_category'].value_counts()

luxury_category
Luxury      6581
Standard    4982
Budget       874
Name: count, dtype: int64

In [28]:
df['floor_category'].value_counts()

floor_category
2.0    5854
1.0    3850
0.0    2733
Name: count, dtype: int64

In [29]:
df['floor_category'] = df['floor_category'].replace({0.0:'Low Floor',1.0:'Mid Floor',2.0:'High Floor'})

In [30]:
df['floor_category'].value_counts()

floor_category
High Floor    5854
Mid Floor     3850
Low Floor     2733
Name: count, dtype: int64

In [31]:
df['Furnishing'].value_counts()

Furnishing
1.0    9686
0.0    2220
2.0     531
Name: count, dtype: int64

In [32]:
status_mapping_Furnishing = {
    0 :'Unfurnished',
    2 : 'Furnished',
    1: 'Semifurnished',

}

df['Furnishing'] = df['Furnishing'].replace(status_mapping_Furnishing)

In [33]:
df['Furnishing'].value_counts()

Furnishing
Semifurnished    9686
Unfurnished      2220
Furnished         531
Name: count, dtype: int64

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12437 entries, 0 to 12436
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Facing                 12437 non-null  object 
 1   No_Bedroom             12437 non-null  int64  
 2   No_Bathroom            12437 non-null  float64
 3   No_Balcony             12437 non-null  object 
 4   Corner_Property        12437 non-null  object 
 5   Furnishing             12437 non-null  object 
 6   study room             12437 non-null  int64  
 7   servant room           12437 non-null  int64  
 8   store room             12437 non-null  int64  
 9   pooja room             12437 non-null  int64  
 10  Overlooking_main road  12437 non-null  int64  
 11  Overlooking_club       12437 non-null  int64  
 12  Overlooking_pool       12437 non-null  int64  
 13  Overlooking_NA         12437 non-null  int64  
 14  flat_age               12437 non-null  object 
 15  se

In [35]:
df.columns

Index(['Facing', 'No_Bedroom', 'No_Bathroom', 'No_Balcony', 'Corner_Property',
       'Furnishing', 'study room', 'servant room', 'store room', 'pooja room',
       'Overlooking_main road', 'Overlooking_club', 'Overlooking_pool',
       'Overlooking_NA', 'flat_age', 'sector_num', 'Super Built-up Area',
       'luxury_category', 'floor_category', 'price'],
      dtype='object')

In [36]:
df.shape


(12437, 20)

In [37]:
df['sector_num'] = df['sector_num'].astype('object')

In [38]:
df.head()

,Facing,No_Bedroom,No_Bathroom,No_Balcony,Corner_Property,Furnishing,study room,servant room,store room,pooja room,Overlooking_main road,Overlooking_club,Overlooking_pool,Overlooking_NA,flat_age,sector_num,Super Built-up Area,luxury_category,floor_category,price
0,West,2,2.0,1,No,Unfurnished,0,0,1,0,1,0,0,0,Relatively New,90,690.0,Luxury,Low Floor,0.61
1,South,2,2.0,1,No,Unfurnished,0,0,1,0,1,0,0,0,Relatively New,90,690.0,Luxury,High Floor,0.70
2,North-East,2,2.0,1,Yes,Semifurnished,0,0,0,0,1,1,0,0,Relatively New,70,836.0,Luxury,High Floor,0.71
3,East,2,2.0,2,Yes,Semifurnished,0,0,0,0,1,1,1,0,New Property,68,1260.0,Luxury,High Floor,1.55
4,North,2,2.0,3+,Yes,Semifurnished,0,0,0,0,0,1,1,0,Relatively New,107,1366.0,Luxury,Mid Floor,1.30


In [39]:
df = df[df['price'] <= 20]

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12360 entries, 0 to 12436
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Facing                 12360 non-null  object 
 1   No_Bedroom             12360 non-null  int64  
 2   No_Bathroom            12360 non-null  float64
 3   No_Balcony             12360 non-null  object 
 4   Corner_Property        12360 non-null  object 
 5   Furnishing             12360 non-null  object 
 6   study room             12360 non-null  int64  
 7   servant room           12360 non-null  int64  
 8   store room             12360 non-null  int64  
 9   pooja room             12360 non-null  int64  
 10  Overlooking_main road  12360 non-null  int64  
 11  Overlooking_club       12360 non-null  int64  
 12  Overlooking_pool       12360 non-null  int64  
 13  Overlooking_NA         12360 non-null  int64  
 14  flat_age               12360 non-null  object 
 15  sector_

In [41]:
df.to_csv('gurgaon_properties_post_feature_selection_v2.csv')

In [42]:
df['price'].describe()

count    12360.000000
mean         3.209152
std          2.456234
min          0.160000
25%          1.750000
50%          2.450000
75%          3.890000
max         20.000000
Name: price, dtype: float64

In [43]:
X = df.drop(columns=['price'])
y = df['price']

In [44]:
y_transformed = np.log1p(y)

In [45]:
np.log1p(5)

1.791759469228055

In [46]:
np.expm1(1.791759469228055)

5.0

In [47]:
import category_encoders as ce

columns_to_encode = ['Facing','Corner_Property', 'No_Balcony', 'flat_age', 'Furnishing', 'luxury_category', 'floor_category']

preprocessor = ColumnTransformer(
    transformers = [
        ('num' , StandardScaler(), ['No_Bedroom', 'No_Bathroom', 'study room', 'servant room', 'store room', 'pooja room','Overlooking_main road', 'Overlooking_club','Overlooking_pool', 'Overlooking_NA', 'Super Built-up Area']),
        ('cat' , OrdinalEncoder(), columns_to_encode),
        ('target_enc', ce.TargetEncoder(), ['sector_num'])

    ],
    remainder = 'passthrough'
)

In [48]:
from sklearn.ensemble import RandomForestRegressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [49]:
from sklearn.metrics import mean_squared_error, r2_score
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
pipeline.fit(X_train,y_train)
    
y_pred = pipeline.predict(X_test)
    
y_pred = np.expm1(y_pred)
    
mae = mean_absolute_error(np.expm1(y_test),y_pred)
r2 = r2_score(np.expm1(y_test),y_pred)
print("Mean Absolute Error (MAE):", mae)
print("R^2 Score:", r2)

Mean Absolute Error (MAE): 0.3909501646368445
R^2 Score: 0.9012791209971859


In [50]:
def scorer(model_name, model):
    output = []
    output.append(model_name)

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)
    
    try:
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        y_pred = np.expm1(y_pred)

        output.append(r2_score(np.expm1(y_test), y_pred))
        output.append(mean_absolute_error(np.expm1(y_test), y_pred))
    except ValueError as e:
        print(f"Error with model {model_name}: {e}")
        output.append(None)
        output.append(None)

    return output

In [51]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from ngboost import NGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

model_dict = {
    'linear_reg': LinearRegression(),
    'svr': SVR(),
    'ridge': Ridge(),
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'extra_trees': ExtraTreesRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost': XGBRegressor(),
    'lightgbm': LGBMRegressor(),    
    #'ngboost': NGBRegressor(), 
    'hist_gradient_boosting': HistGradientBoostingRegressor(),  
    'bagging': BaggingRegressor(), 
    'kernel_ridge': KernelRidge(),     
}


In [52]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001925 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 9888, number of used features: 19
[LightGBM] [Info] Start training from score 1.314906


In [53]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [54]:
model_df.sort_values(['mae'])

,name,r2,mae
4,random_forest,0.901665,0.389366
12,bagging,0.888226,0.416266
5,extra_trees,0.890925,0.416283
9,xgboost,0.903004,0.418389
11,hist_gradient_boosting,0.887244,0.472199
10,lightgbm,0.887588,0.472676
3,decision_tree,0.807745,0.498421
6,gradient_boosting,0.851756,0.552469
1,svr,0.815345,0.609907
8,mlp,0.812433,0.628215


In [55]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from sklearn.datasets import make_regression
import os
from sklearn.model_selection import train_test_split

In [57]:
import warnings
warnings.filterwarnings("ignore")

def objective(params):
    model = XGBRegressor(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        min_child_weight=int(params['min_child_weight']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        max_delta_step=int(params['max_delta_step']),
        scale_pos_weight=params['scale_pos_weight'],
        max_bin=int(params['max_bin']),
        grow_policy=params['grow_policy'],
        booster='gbtree',
        n_jobs=-1,
    )

    pipeline = Pipeline([
        ('preprocessor', preprocessor), 
        ('regressor', model)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_pred = np.expm1(y_pred)    
    mae = mean_absolute_error(np.expm1(y_test),y_pred)   
    return {'loss': mae, 'status': STATUS_OK}

param_space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 5000, 20000, 10)),
    'max_depth': scope.int(hp.quniform('max_depth', 2, 12, 1)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.05)),
    'subsample': hp.uniform('subsample', 0.6, 0.9),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
    'min_child_weight': scope.int(hp.quniform('min_child_weight', 1, 4, 1)),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'reg_alpha': hp.uniform('reg_alpha', 0.1, 0.3),
    'reg_lambda': hp.uniform('reg_lambda', 0.1, 0.3),
    'max_delta_step': scope.int(hp.quniform('max_delta_step', 3, 6, 1)),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 8, 10),
    'max_bin': scope.int(hp.quniform('max_bin', 300, 400, 1)),
    'grow_policy': hp.choice('grow_policy', ['depthwise', 'lossguide']),
}

trials = Trials()
max_evals = 80

with tqdm(total=max_evals, desc="Hyperparameter Tuning") as pbar:
    best_params = fmin(
        fn=objective,
        space=param_space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=np.random.default_rng(42),
        show_progressbar=True
    )

print("\nBest Hyperparameters:", best_params)

final_model = XGBRegressor(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    min_child_weight=int(best_params['min_child_weight']),
    gamma=best_params['gamma'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    max_delta_step=int(best_params['max_delta_step']),
    scale_pos_weight=best_params['scale_pos_weight'],
    max_bin=int(best_params['max_bin']),
    grow_policy=['depthwise', 'lossguide'][best_params['grow_policy']],
    booster='gbtree',
    n_jobs=-1,
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', final_model)
])


pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nPerformance of the Final Model:")
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R^2 Score:", r2)



Hyperparameter Tuning:   0%|                                                                    | 0/80 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████| 80/80 [11:21<00:00,  8.52s/trial, best loss: 0.3818393713078486]

Hyperparameter Tuning:   0%|                                                                    | 0/80 [11:21<?, ?it/s]



Best Hyperparameters: {'colsample_bytree': 0.7450117923076128, 'gamma': 6.942715290022107e-06, 'grow_policy': 1, 'learning_rate': 0.04431254560995843, 'max_bin': 368.0, 'max_delta_step': 3.0, 'max_depth': 6.0, 'min_child_weight': 1.0, 'n_estimators': 7540.0, 'reg_alpha': 0.1795482968359151, 'reg_lambda': 0.2647015630918937, 'scale_pos_weight': 9.388268962794474, 'subsample': 0.6972729354682095}



Performance of the Final Model:
Mean Squared Error (MSE): 0.014506833026846208
Mean Absolute Error (MAE): 0.08102135293295862
R^2 Score: 0.9338538035309351


In [58]:
import category_encoders as ce

columns_to_encode = ['Facing','Corner_Property', 'No_Balcony', 'flat_age', 'Furnishing', 'luxury_category', 'floor_category']

preprocessor = ColumnTransformer(
    transformers = [
        ('num' , StandardScaler(), ['No_Bedroom', 'No_Bathroom', 'study room', 'servant room', 'store room', 'pooja room','Overlooking_main road', 'Overlooking_club','Overlooking_pool', 'Overlooking_NA', 'Super Built-up Area']),
        ('cat' , OrdinalEncoder(), columns_to_encode),
        ('target_enc', ce.TargetEncoder(), ['sector_num'])

    ],
    remainder = 'passthrough'
)

In [59]:
X.shape

(12360, 19)

In [60]:
y_transformed.shape

(12360,)

In [61]:
X.columns

Index(['Facing', 'No_Bedroom', 'No_Bathroom', 'No_Balcony', 'Corner_Property',
       'Furnishing', 'study room', 'servant room', 'store room', 'pooja room',
       'Overlooking_main road', 'Overlooking_club', 'Overlooking_pool',
       'Overlooking_NA', 'flat_age', 'sector_num', 'Super Built-up Area',
       'luxury_category', 'floor_category'],
      dtype='object')

In [62]:
pipeline = Pipeline([
        ('preprocessor', preprocessor), 
        ('regressor', final_model)
    ])

In [63]:
pipeline.fit(X[:12350],y_transformed[:12350])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['No_Bedroom', 'No_Bathroom',
                                                   'study room', 'servant room',
                                                   'store room', 'pooja room',
                                                   'Overlooking_main road',
                                                   'Overlooking_club',
                                                   'Overlooking_pool',
                                                   'Overlooking_NA',
                                                   'Super Built-up Area']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['Facing', 'Corner_Property',
                                                   'No_Balc...
                              grow_policy='lossguide', importance_type=None,
                              interaction_constraints=None,
                              learning_rate=0.04431254560995843, max_bin=368,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=3, max_depth=6, max_leaves=None,
                              min_child_weight=1, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=7540, n_jobs=-1,
                              num_parallel_tree=None, random_state=None, ...))])

In [64]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [65]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [66]:
X['Furnishing'].value_counts()

Furnishing
Semifurnished    9624
Unfurnished      2213
Furnished         523
Name: count, dtype: int64

In [67]:
X['Facing'].unique()

array(['West', 'South', 'North-East', 'East', 'North', 'South-East',
       'South-West', 'North-West'], dtype=object)

In [68]:
X['flat_age'].value_counts()

flat_age
Relatively New        5415
New Property          2255
Moderately Old        2098
Under Construction    1859
Old Property           729
Undefined                4
Name: count, dtype: int64

In [69]:
X['floor_category'].value_counts()

floor_category
High Floor    5819
Mid Floor     3832
Low Floor     2709
Name: count, dtype: int64

In [70]:
X.iloc[12352]

Facing                             East
No_Bedroom                            3
No_Bathroom                         4.0
No_Balcony                           3+
Corner_Property                     Yes
Furnishing                Semifurnished
study room                            1
servant room                          1
store room                            0
pooja room                            1
Overlooking_main road                 1
Overlooking_club                      1
Overlooking_pool                      1
Overlooking_NA                        0
flat_age                 Relatively New
sector_num                           81
Super Built-up Area              2660.0
luxury_category                  Luxury
floor_category                Low Floor
Name: 12429, dtype: object

In [71]:
data = [['East', 3 , 4.0 , '3+', 'Yes', 'Semifurnished', 1, 1,0, 1 ,1 ,1 , 1 , 0, 'Relatively New' , 81 , 2660.0 , 'Luxury' , 'Low Floor'  ]]
columns = ['Facing', 'No_Bedroom', 'No_Bathroom', 'No_Balcony', 'Corner_Property',
       'Furnishing', 'study room', 'servant room', 'store room', 'pooja room',
       'Overlooking_main road', 'Overlooking_club', 'Overlooking_pool',
       'Overlooking_NA', 'flat_age', 'sector_num', 'Super Built-up Area',
       'luxury_category', 'floor_category']

one_df = pd.DataFrame(data, columns=columns)

one_df

,Facing,No_Bedroom,No_Bathroom,No_Balcony,Corner_Property,Furnishing,study room,servant room,store room,pooja room,Overlooking_main road,Overlooking_club,Overlooking_pool,Overlooking_NA,flat_age,sector_num,Super Built-up Area,luxury_category,floor_category
0,East,3,4.0,3+,Yes,Semifurnished,1,1,0,1,1,1,1,0,Relatively New,81,2660.0,Luxury,Low Floor


In [72]:
np.expm1(pipeline.predict(one_df))

array([3.506143], dtype=float32)

In [73]:
np.expm1(y_transformed.iloc[12352])

3.4000000000000004